In [42]:
import pandas as pd
import itertools
import plotly.express as px
import plotly.graph_objects as go
import matplotlib.pyplot as plt
import numpy as np
import mygene

Modified from de_combinations.ipynb
Find mutation status for canonical genes in any gene expression list, based on threshold: 


In [46]:
subtypes = pd.read_csv('GEO_data/gse72970_collapsed_cmscaller_subtype.csv')
subtypes.rename(columns = {'Unnamed: 0':'case_id'}, inplace=True)
data = pd.read_csv('GEO_data/GSE72970_genelevelexpression_matrix.csv')
data.rename(columns = {'Unnamed: 0':'entrez_id'}, inplace=True)
#data = data.drop_duplicates('entrez_id', keep = 'last') #drop duplicates of entrez_ids

In [47]:
subtypes.head()

,case_id,prediction,d.CMS1,d.CMS2,d.CMS3,d.CMS4,p.value,FDR
0,GSM1875897,NaN,0.707792,0.704436,0.700065,0.869283,1.000000,1.000000
1,GSM1875898,NaN,0.757000,0.688300,0.699422,0.882335,1.000000,1.000000
2,GSM1875899,NaN,0.712476,0.675538,0.741492,0.762567,0.884116,0.970180
3,GSM1875900,CMS4,0.733731,0.654896,0.754345,0.620548,0.011988,0.015814
4,GSM1875901,CMS4,0.701315,0.709047,0.764964,0.558683,0.001000,0.001409


In [48]:
print(np.shape(data))
data.head()

(35411, 125)


,entrez_id,GSM1875897,GSM1875898,GSM1875899,GSM1875900,GSM1875901,GSM1875902,GSM1875903,GSM1875904,GSM1875905,...,GSM1876011,GSM1876012,GSM1876013,GSM1876014,GSM1876015,GSM1876016,GSM1876017,GSM1876018,GSM1876019,GSM1876020
0,10,8.293985,7.843415,6.938467,7.215872,6.391727,7.619402,9.378036,6.397840,8.840072,...,7.368687,9.321897,7.711947,9.251508,5.427702,5.566483,8.903231,5.652385,9.499557,5.289448
1,100,8.163994,6.916187,7.756554,8.415470,7.793665,7.417628,7.483132,7.392226,7.740027,...,6.740413,7.033339,7.974428,6.044235,6.836513,6.216799,7.439033,8.589182,8.017036,8.961782
2,1000,6.216392,6.392460,6.113381,5.710079,7.121346,6.517431,6.309919,6.085059,5.550282,...,5.531246,6.051520,6.265509,5.734785,5.540122,5.903405,5.632783,5.346066,5.354300,5.380150
3,10000,5.982941,5.276512,6.267585,7.406841,8.177597,6.892591,7.908049,6.808414,6.967851,...,7.145327,7.768157,8.455111,5.870275,8.266770,7.203759,6.975459,8.695998,7.516984,7.792660
4,100009676,5.521854,6.403854,5.836684,6.070767,4.361525,5.699826,4.936783,5.722869,5.625479,...,5.420684,5.610872,5.460612,5.799970,5.427722,5.531225,5.282096,5.186960,5.424344,5.435326


In [49]:
entrezid = data['entrez_id'].values

In [50]:
def mygene_query_hugo2entrez(gene_list):
    mg = mygene.MyGeneInfo()
    out = mg.querymany(gene_list, scopes='symbol', fields='entrezgene', species='human')
    return out

In [51]:
low_genes = ['KRAS', 'APC', 'DCC', 'TFGBR2', 'SMAD2', 'SMAD4', 'BAX', 'TP53', 'MLH1', 'MLH2']
high_genes = ['MLH3', 'BRAF', 'PIK3CA', 'POU2AF3', 'GALNT12'] #615694 = POU2AF3 = COLCA2 https://genome.ucsc.edu/cgi-bin/hgc?hgsid=1317728599_8W1TAZ1yc1qca0Q8ZPdCZNdW2L9u&db=hg38&c=chr11&l=111293388&r=111305048&o=111298545&t=111308735&g=omimGene2&i=615694

In [52]:
#edit: deal with symbols that are not found instead of manually addressing this issue

out = mygene_query_hugo2entrez(high_genes)
entrez_id = []
for query in range(len(out)): 
    if ('entrezgene' in out[query]):
        e = str(out[query]['entrezgene'])
        h = out[query]['query']
        entrez_id.append((h,e))


querying 1-5...done.
Finished.


In [53]:
high_genes_df = pd.DataFrame(entrez_id, columns= ['query', 'entrez_id'])
high_genes_df['expr'] = 'high'

In [54]:
#edit: deal with symbols that are not found instead of manually addressing this issue

mygene_query_hugo2entrez(low_genes)
entrez_id = []
for query in range(len(out)): 
    if ('entrezgene' in out[query]):
        e = str(out[query]['entrezgene'])
        h = out[query]['query']
        entrez_id.append((h,e))


querying 1-10...done.
Finished.
2 input query terms found no hit:
	['TFGBR2', 'MLH2']
Pass "returnall=True" to return complete lists of duplicate or missing query terms.


In [55]:
#manually add missing hits: https://www.genecards.org/cgi-bin/carddisp.pl?gene=TGFBR2
entrez_id.append(('TFGBR2', 7048)) #can't find entrez id for MLH2?
low_genes_df = pd.DataFrame(entrez_id, columns= ['query', 'entrez_id'])
low_genes_df['expr'] = 'low'

In [56]:
colotype_genelist = pd.concat([low_genes_df, high_genes_df], ignore_index = True)

In [57]:
colotype_genelist

,query,entrez_id,expr
0,MLH3,27030,low
1,BRAF,673,low
2,PIK3CA,5290,low
3,POU2AF3,120376,low
4,GALNT12,79695,low
5,TFGBR2,7048,low
6,MLH3,27030,high
7,BRAF,673,high
8,PIK3CA,5290,high
9,POU2AF3,120376,high


In [58]:
sub = colotype_genelist['entrez_id'].values

In [59]:
len(sub)

11

In [60]:
canon_gene_exp = data[data['entrez_id'].isin(sub)]
canon_gene_entrez_id = canon_gene_exp['entrez_id'].values
canon_gene_exp_dataonly = canon_gene_exp.drop('entrez_id', axis = 1)
canon_gene_exp_dataonly_copy = canon_gene_exp_dataonly.copy()

In [61]:
for col in canon_gene_exp_dataonly.columns:
    canon_gene_exp_dataonly_copy.loc[canon_gene_exp_dataonly[col] > canon_gene_exp_dataonly[col].mean() * 1.4, col] = 'high'
    canon_gene_exp_dataonly_copy.loc[canon_gene_exp_dataonly[col] < canon_gene_exp_dataonly[col].mean() * 0.7, col] = 'low'



In [62]:
canon_gene_exp_dataonly_copy['entrez_id'] = canon_gene_entrez_id

In [63]:
mg = mygene.MyGeneInfo()
out = mg.querymany(canon_gene_entrez_id, scopes='entrezgene', fields='symbol', species='human')

hugo = []
for query in range(len(out)): 
    if ('symbol' in out[query]):
        h = str(out[query]['symbol'])
        e = out[query]['query']
        hugo.append((h))
        
canon_gene_exp_dataonly_copy['HGNC'] = hugo



querying 1-5...done.
Finished.


In [65]:
df = canon_gene_exp_dataonly_copy
allowed_vals = ['high', 'low']
df[~df.isin(allowed_vals)] = "WT"

In [68]:
df['entrez_id'] = canon_gene_entrez_id
df['HGNC'] = hugo
df

,GSM1875897,GSM1875898,GSM1875899,GSM1875900,GSM1875901,GSM1875902,GSM1875903,GSM1875904,GSM1875905,GSM1875906,...,GSM1876013,GSM1876014,GSM1876015,GSM1876016,GSM1876017,GSM1876018,GSM1876019,GSM1876020,entrez_id,HGNC
2984,WT,WT,WT,WT,low,low,WT,WT,WT,WT,...,low,low,WT,WT,WT,WT,WT,WT,120376,POU2AF3
23186,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,27030,MLH3
27363,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,5290,PIK3CA
30906,WT,WT,WT,WT,WT,WT,WT,WT,WT,WT,...,WT,WT,WT,WT,WT,WT,WT,WT,673,BRAF
32083,WT,WT,WT,WT,WT,high,WT,high,WT,WT,...,WT,high,WT,WT,WT,WT,WT,WT,79695,GALNT12


In [71]:
df.to_csv('GEO_canongene_masked_boolean.csv')

In [72]:
df.T.to_csv('GEO_canongene_masked_boolean_T.csv')

To do: 
- Why are only 5 of the canonical genes showing up with the microarray data?
- Revisit how collapseRows works (see cmscaller_R_multidata.ipynb)

,2984,23186,27363,30906,32083
GSM1875897,WT,WT,WT,WT,WT
GSM1875898,WT,WT,WT,WT,WT
GSM1875899,WT,WT,WT,WT,WT
GSM1875900,WT,WT,WT,WT,WT
GSM1875901,low,WT,WT,WT,WT
...,...,...,...,...,...
GSM1876018,WT,WT,WT,WT,WT
GSM1876019,WT,WT,WT,WT,WT
GSM1876020,WT,WT,WT,WT,WT
entrez_id,120376,27030,5290,673,79695
